# Tarea - Modelo ARIMA

Sofía Maldonado García 

11/02/2026

### 1) Utilizando como base la forma canónica de un modelo ARIMA (p,d,q) desarrollar el caso en el que se considera un modelo de orden ARIMA(2,1,2)
- Realizar el procedimiento y detalle de sus conclusiones/pasos.

Un modelo ARIMA(2,1,2) representa un caso en el que:
- ($p=2$) Los dos valores anteriores tienen influencia significativa en el valor actual
- ($d=1$) Se hace una sola diferenciación para hacer que la serie de tiempo sea estacionaria
- ($q=2$) Los dos errores anteriores son relevantes en el valor actual.

Un caso donde se pudiese usar un modelo como este sería en [insertar aquí]

### 2) Realizar un modelo para pronóstico de una serie de tiempo (el estudiante decide qué datos usar) utilizando 3 librerías de forecast en Python.
- Validar si su modelo se realiza con una serie de tiempo estacionaria o no
- Graficar serie de tiempo utilizando la librería plotly
- Comprobar los resultados obtenidos en las diferentes librerías

In [4]:
# Imports
# Generales
import numpy as np
import pandas as pd



ModuleNotFoundError: No module named 'pandas'

In [5]:
%pip install pandas

^C
ERROR: Operation cancelled by user
Note: you may need to restart the kernel to use updated packages.
